## Preprocess the Sim2Real dataset

The Sim2Real dataset uses jpeg images to represent wildfire masks. This representation is very lightweight but slow to work with. 
To benchmark faster, we convert them to binary NumPy files. These files are ~100 times heavier and it requires an overhead time to convert from jpeg to them, but once they are created and stored, they allow 50--100x faster operations.

1. Download the dataset from [Sim2Real-Fire GitHub repository](https://github.com/TJU-IDVLab/Sim2Real-Fire).
2. Extract all files from the dataset into your `Dataset` folder
3. Run the preprocessing function to convert from the jpeg representation to the NumPy one, and compute burn maps

In [1]:
import sys
print(sys.modules.keys())
print(sys.path)



dict_keys(['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '_warnings', '_weakref', '_io', 'marshal', 'posix', '_frozen_importlib_external', 'time', 'zipimport', '_codecs', 'codecs', 'encodings.aliases', 'encodings', 'encodings.utf_8', '_signal', '_abc', 'abc', 'io', '__main__', '_stat', 'stat', '_collections_abc', 'genericpath', 'posixpath', 'os.path', 'os', '_sitebuiltins', '_distutils_hack', 'site', 'importlib._bootstrap', 'importlib._bootstrap_external', 'warnings', 'importlib', 'importlib.machinery', 'importlib._abc', 'itertools', 'keyword', '_operator', 'operator', 'reprlib', '_collections', 'collections', 'types', '_functools', 'functools', '_weakrefset', 'threading', 'importlib.util', 'runpy', 'enum', '_sre', 're._constants', 're._parser', 're._casefix', 're._compiler', 'copyreg', 're', 'fnmatch', 'ntpath', 'errno', 'urllib', 'math', 'ipaddress', 'urllib.parse', 'pathlib', 'collections.abc', 'contextlib', '_typing', 'typing.io', 'typing.re', 'typing', 'ipykernel._ver

In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install shapely


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [1]:
# import requred modules
import sys
import os
import time
import numpy as np

# Add code to path
module_path = os.path.abspath(".") + "/code"
if module_path not in sys.path:
    sys.path.append(module_path)

from dataset import preprocess_sim2real_dataset, load_scenario_npy, compute_and_save_burn_maps_sim2real_dataset
from Strategy import RandomDroneRoutingStrategy, return_no_custom_parameters, SensorPlacementOptimization, RandomSensorPlacementStrategy, LoggedOptimizationSensorPlacementStrategy,DroneRoutingOptimizationSlow, DroneRoutingOptimizationModelReuse, DroneRoutingOptimizationModelReuseIndex, LoggedDroneRoutingStrategy, LogWrapperDrone, LogWrapperSensor, DroneRoutingOptimizationModelReuseIndexRegularized
from benchmark import run_benchmark_scenario,run_benchmark_scenarii_sequential, get_burnmap_parameters,run_benchmark_scenarii_sequential_precompute, benchmark_on_sim2real_dataset_precompute
from displays import create_scenario_video
from new_clustering import get_wrapped_strategy
from wrappers import wrap_log_sensor_strategy, wrap_log_drone_strategy

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython
Initializing the Julia session. This can take up to 1 minute.
initializing the ground sensor julia module
installing packages
initializing the drone julia module
Julia session initialized.
=== TEST PRINT: Entered RandomPlacementStrategy class definition ===
=== TEST PRINT: Entered DroneRoutingOptimizationModelReuseIndex class definition ===
=== TEST PRINT: Entered DroneRoutingOptimizationModelReuseIndex class definition ===


Uncomment the cell below to preprocess the dataset.
- `n_max_scenarii_per_layout` controls the number of scenarios we convert from jpeg to NumPy files for each layout.
- If executed on all ~50 layouts of the dataset, the code below takes ~15mins to run and generates **400 GB** of data. Make sure to have the space.

In [2]:
# preprocess_sim2real_dataset("WideDataset/", n_max_scenarii_per_layout=100) 

We can now run the benchmark function on any scenario.
1. We load the scenario using `load_scenario_npy` since it is in preprocessed `npy` format
2. Werun the benchmark using `run_benchmark_scenario` that takes as input:
    - The `scenario`, 
    - The sensor placement strategy and the drone routing strategy
    - A dictionary `custom_initialization_parameters` that contains any custom initialization inputs for your strategy functions (such as the burn map if your strategy needs a burn map as input)
    - A python function `custom_step_parameters_function` that returns a dictionary of custom inputs for your routing function. This function is executed by the benchmarking code at each time step. Your routing function will be called with: `routing(custom_step_parameters_function())` internally

In [2]:
# change values here to change benchmarking parameters
def my_automatic_layout_parameters(scenario:np.ndarray,b,c):
    return {
        "N": scenario.shape[1],
        "M": scenario.shape[2],
        "max_battery_distance": -1,
        "max_battery_time": 10,
        "n_drones": 20,
        "n_ground_stations": 250,
        "n_charging_stations": 16,
        #tau
        #reg_ground
        #reg_charging
        #reg_ground_charging
        #cap_drones
    }

In [3]:
# That's very fast to run!
print("starting benchmark")
time_start = time.time()
scenario = load_scenario_npy("Dataset/0001/scenarii/0001_00002.npy")
results, (position_history, ground, charging)  = run_benchmark_scenario(scenario, SensorPlacementOptimization, get_wrapped_strategy(DroneRoutingOptimizationModelReuseIndex), custom_initialization_parameters = {"burnmap_filename": "./Dataset/0001/burn_map.npy", "load_from_logfile": False, "reevaluation_step": 6, "optimization_horizon":6, "regularization_param": 0.0001}, custom_step_parameters_function = return_no_custom_parameters, automatic_initialization_parameters_function=my_automatic_layout_parameters, return_history=True)
print(results)




starting benchmark
calling julia optimization model
NEW STRATEGY 3 - Optimized for speed
Number of cells discarded: 29962
Set parameter Username
Set parameter LicenseID to value 2630735
Academic license - for non-commercial use only - expires 2026-03-03
Took 1.2672777082771063 seconds to create model
optimization finished
Charging Station Locations from Julia Optimization Model:  []
ground sensor locations
[(33, 108), (34, 108), (35, 108), (36, 108), (37, 108), (31, 109), (32, 109), (34, 109), (35, 109), (36, 109), (32, 110), (34, 110), (35, 110), (36, 110), (37, 110), (31, 111), (34, 111), (47, 114), (46, 115), (47, 115), (49, 115), (50, 115), (47, 116), (48, 116), (49, 116), (51, 116), (48, 117), (49, 117), (50, 117), (52, 117), (51, 118), (52, 118), (53, 118), (52, 119), (53, 119), (51, 120), (52, 120), (53, 120), (54, 120), (52, 121), (54, 121), (25, 127), (24, 128), (25, 128), (26, 128), (27, 128), (34, 128), (35, 128), (37, 128), (25, 129), (26, 129), (27, 129), (28, 129), (29, 1

In [ ]:
def custom_initialization_parameters_function(input_dir:str):
    print(f"input_dir: {input_dir}")
    return {"burnmap_filename": f"{'/'.join(input_dir.strip('/').split('/')[:-1])}/burn_map.npy", "reevaluation_step": 5, "optimization_horizon":5}
#run_benchmark_scenarii_sequential_precompute(input_dir="WideDataset/0001/scenarii/", sensor_placement_strategy=LogWrapperSensor, drone_routing_strategy=LogWrapperDrone, custom_initialization_parameters_function=custom_initialization_parameters_function, custom_step_parameters_function = return_no_custom_parameters, starting_time=0, max_n_scenarii=100)
benchmark_on_sim2real_dataset_precompute("Dataset/", LogWrapperSensor, LogWrapperDrone, custom_initialization_parameters_function, return_no_custom_parameters, max_n_scenarii=100, starting_time=0)
    

In [5]:
create_scenario_video(scenario[:len(position_history)],drone_locations_history=position_history,starting_time=0,out_filename='test_simulation', ground_sensor_locations = ground, charging_stations_locations = charging)

AttributeError: 'function' object has no attribute 'cluster_data'

We can visualize the strategy in action by creating a video of the scenario. 
1. Use the `return_history` parameter or `run_benchmark_scenario` to output the log of ground sensor and drone positions during the benchmark
2. Use the `create_scenario_video` function to compile the video. This can take a couple seconds

In [4]:
delta , device , (position_history, ground, charging)  = run_benchmark_scenario(scenario, SensorPlacementOptimization, RandomDroneRoutingStrategy, custom_initialization_parameters = {"burnmap_filename": "./Dataset/0001/burn_map.npy"}, custom_step_parameters_function = return_no_custom_parameters, automatic_initialization_parameters_function=my_automatic_layout_parameters, return_history=True)
create_scenario_video(scenario[:len(position_history)],drone_locations_history=position_history,starting_time=0,out_filename='test_simulation', ground_sensor_locations = ground, charging_stations_locations = charging)

calling julia optimization model
In julia fuinction
Set parameter Username
Set parameter LicenseID to value 2612529
Academic license - for non-commercial use only - expires 2026-01-21
Took 8.259267374873161 seconds
optimization finished
Video saved at: display_test_simulation/test_simulation.mp4


Instead of running a benchmark on a single scenario, we are interested in running the benchmark on all scenarii of a given layout (potentially in parallel!)

1. We use `run_benchmark_scenarii_sequential`

In [3]:
run_benchmark_scenarii_sequential("MinimalDataset/0001/scenarii/", RandomSensorPlacementStrategy, RandomDroneRoutingStrategy, custom_initialization_parameters_function = get_burnmap_parameters, custom_step_parameters_function = return_no_custom_parameters, starting_time=0, max_n_scenarii=3)
run_benchmark_scenarii_sequential("MinimalDataset/0001/Satellite_Images_Mask/", RandomSensorPlacementStrategy, RandomDroneRoutingStrategy, custom_initialization_parameters_function = get_burnmap_parameters, custom_step_parameters_function = return_no_custom_parameters, starting_time=0, max_n_scenarii=3, file_format="jpg")

100%|██████████| 3/3 [00:00<00:00, 91.85it/s]


Processing scenario MinimalDataset/0001/scenarii/0001_00033.npy
Processing scenario MinimalDataset/0001/scenarii/0001_00034.npy
Processing scenario MinimalDataset/0001/scenarii/0001_00005.npy
Avg time steps to fire detection: 0.0
Fire found 0.0% of the time by ground sensor
Fire found 0.0% of the time by charging station
Fire found 0.0% of the time by drone
Fire found 100.0% of the time by undetected
Avg routing execution time: 0.0000 sec
Avg fire size at detection (cells): 0.00
Avg fire size at detection (% of map): 0.00%
Avg percentage of map explored by drones: 0.20%
Avg total distance traveled by drones: 357.33 units
Avg drone entropy per timestep: 0.0000
Avg sensor placement entropy: 0.0000


100%|██████████| 3/3 [00:00<00:00, 23.72it/s]

Processing scenario MinimalDataset/0001/Satellite_Images_Mask/0001_00005
Processing scenario MinimalDataset/0001/Satellite_Images_Mask/0001_00002
Processing scenario MinimalDataset/0001/Satellite_Images_Mask/0001_00003
Avg time steps to fire detection: 0.0
Fire found 0.0% of the time by ground sensor
Fire found 0.0% of the time by charging station
Fire found 0.0% of the time by drone
Fire found 100.0% of the time by undetected
Avg routing execution time: 0.0000 sec
Avg fire size at detection (cells): 0.00
Avg fire size at detection (% of map): 0.00%
Avg percentage of map explored by drones: 0.20%
Avg total distance traveled by drones: 373.33 units
Avg drone entropy per timestep: 0.0000
Avg sensor placement entropy: 0.0000


On all of the dataset!

In [ ]:
benchmark_on_sim2real_dataset("WideDataset/", SensorPlacementStrategy, DroneRoutingStrategy, custom_initialization_parameters_function = lambda x: None, custom_step_parameters_function = return_no_custom_parameters, max_n_scenarii=100, starting_time=0)

In [4]:
# That's very fast to run!
time_start = time.time()
scenario = load_scenario_npy("WideDataset/0001/scenarii/0001_00002.npy")
device, delta_t, _ = run_benchmark_scenario(scenario, RandomSensorPlacementStrategy, DroneRoutingOptimizationSlow, custom_initialization_parameters = {"burnmap_filename": "./WideDataset/0001/burn_map.npy", "call_every_n_steps": 5, "optimization_horizon": 10}, custom_step_parameters_function = return_no_custom_parameters, automatic_initialization_parameters_function=my_automatic_layout_parameters, return_history=True)
print("Fire detected in ", delta_t, "time steps by device: ", device)
print(f"Time taken to run benchmark on the scenario: {time.time() - time_start} seconds")


calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling ju

In [13]:
# define function to get parameters for optimal drone routing strategy
# def get_routing_optimization_parameters(input_dir:str):
#     return {
#         "burnmap_filename": f"{input_dir}/burn_map.npy",
#         "call_every_n_steps": 5,
#         "optimization_horizon": 10
#     }

# add the log
